### Employee Scheduling

Example from Googles OR-Tools docs with some notes for clarity. [The original problem is available under this link](https://developers.google.com/optimization/scheduling/employee_scheduling).

We're trying to create a work schedule for four nurses over three days, with these conditions:

- Each day is divided into three 8-hour shifts.
- Every day, each shift is assigned to a single nurse, and no nurse works more than one shift.
- Each nurse is assigned to at least two shifts during the three-day period

This means we don't try to solve for any objective function, instead we are only interested in finding viable solutions.

After finding viable solutions, we will also look at a way of planning for preferred shift assignments.

In [81]:
from ortools.sat.python import cp_model

In [82]:
num_nurses = 4
num_shifts = 3
num_days = 3
all_nurses = range(num_nurses)
all_shifts = range(num_shifts)
all_days = range(num_days)

In [83]:
model = cp_model.CpModel()

### Variables
we create a dict shifts with the key being a tuple describing if nurse n is assigned to shift s on day d.

(1, 1, 1): NewBoolVar("shift_n1_d1_s1"), -> True means Nurse on is working shift 1 on day one

In [84]:
shifts = {}
for n in all_nurses:
    for d in all_days:
        for s in all_shifts:
            shifts[(n, d, s)] = model.NewBoolVar(f"shift_n{n}_d{d}_s{s}")

print({k: shifts[k] for k in list(shifts)[:3]})

{(0, 0, 0): shift_n0_d0_s0(0..1), (0, 0, 1): shift_n0_d0_s1(0..1), (0, 0, 2): shift_n0_d0_s2(0..1)}


### Constraints

The following is very cool! The solver contains some methods which make it very easy to uphold our constraints:

1. Each shift is assigned to a single nurse per day.
2. Each nurse works at most one shift per day.

**Constraint 1** can easily be achieved by using the method model.AddExactlyOne().

For each day and each shift, we add a constraint. The constraint contains that only one of the bool vars within the constraint is allowed to be true. In our case, these bool vars are telling us whether a nurse is assigned to a shift.

**Constraint 2** can be achieved using model.AddAtMostOne()

For each of our nurses, we look at each day. Inside this day, we are only allowed to set one (or less) of the shift assignments to True.



In [85]:
# Each shift is assigned to a single nurse per day.
for d in all_days:
    for s in all_shifts:
        model.AddExactlyOne(shifts[(n, d, s)] for n in all_nurses)

In [86]:
# Each nurse works at most one shift per day. 0 is also a valid number of shifts.
for n in all_nurses:
    for d in all_days:
        model.AddAtMostOne(shifts[(n, d, s)] for s in all_shifts)

We want to assign shifts as evenly as possible. For that, we calculate the minimum number of shifts per nurse.

We then check if the total number of shifts divided by the number of nurses is even. If it is, that means we can assign the same number of shifts to each nurse. If it isn't, we have to add one slack shift to the maximum shifts per nurse, since extra shifts are required.

In [87]:
# The minimum number of shifts per nurse is the total number of shifts divided by the number of nurses.
# Note: // is integer division in Python, so it returns an integer even if the division has a remainder.
min_shifts_per_nurse = (num_shifts * num_days) // num_nurses

# Here we find out if we can assign the shifts evenly or if we need to assign one more shift to some nurses.
if num_shifts * num_days % num_nurses == 0:
    max_shifts_per_nurse = min_shifts_per_nurse
else:
    max_shifts_per_nurse = min_shifts_per_nurse + 1

# Each nurse works between min_shifts_per_nurse and max_shifts_per_nurse shifts.
# For every nurse, we go through all days and all shifts and add the shifts to a list.
# Then we add a constraint that the number of shifts worked by the nurse is between min_shifts_per_nurse and max_shifts_per_nurse.
# the sum of shifts worked can be used since it represents booleans, which are 1 if the shift is worked and 0 if it is not.
for n in all_nurses:
    shifts_worked = []
    for d in all_days:
        for s in all_shifts:
            shifts_worked.append(shifts[(n, d, s)])
    model.Add(min_shifts_per_nurse <= sum(shifts_worked))
    model.Add(sum(shifts_worked) <= max_shifts_per_nurse) # ensures no nurse is assigned more than max_shifts_per_nurse

### Objective

We are not optimizing, but are interested in viable solutions. Therefore, wo don't have an objective function. Instead, we want to get all solutions.

In [88]:
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

### Solution Callback

A solution callback is a way of telling the solver what to do when it finds a solution. In our case, every time we find a solution, we will print it. It follows a certain structure, another example can be found [here](https://developers.google.com/optimization/cp/cp_solver).

In [89]:
class NursesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shifts, num_nurses, num_days, num_shifts, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_nurses = num_nurses
        self._num_days = num_days
        self._num_shifts = num_shifts
        self._solution_count = 0
        self._solution_limit = limit

    # This function is called at each new solution.
    def on_solution_callback(self):
        self._solution_count += 1
        print(f"Solution {self._solution_count}")
        for d in range(self._num_days):
            print(f"Day {d}")
            for n in range(self._num_nurses):
                is_working = False
                for s in range(self._num_shifts):
                    if self.Value(self._shifts[(n, d, s)]):
                        is_working = True
                        print(f"  Nurse {n} works shift {s}")
                if not is_working:
                    print(f"  Nurse {n} does not work")
        if self._solution_count >= self._solution_limit:
            print(f"Stop search after {self._solution_limit} solutions")
            self.StopSearch()

    def solutionCount(self):
        return self._solution_count

# We print two solutions.
solution_limit = 2
solution_printer = NursesPartialSolutionPrinter(
    shifts, num_nurses, num_days, num_shifts, solution_limit
)

In [90]:
solver.Solve(model, solution_printer)

Solution 1
Day 0
  Nurse 0 does not work
  Nurse 1 works shift 0
  Nurse 2 works shift 1
  Nurse 3 works shift 2
Day 1
  Nurse 0 works shift 2
  Nurse 1 does not work
  Nurse 2 works shift 1
  Nurse 3 works shift 0
Day 2
  Nurse 0 works shift 2
  Nurse 1 works shift 1
  Nurse 2 works shift 0
  Nurse 3 does not work
Solution 2
Day 0
  Nurse 0 works shift 0
  Nurse 1 does not work
  Nurse 2 works shift 1
  Nurse 3 works shift 2
Day 1
  Nurse 0 does not work
  Nurse 1 works shift 2
  Nurse 2 works shift 1
  Nurse 3 works shift 0
Day 2
  Nurse 0 works shift 2
  Nurse 1 works shift 1
  Nurse 2 works shift 0
  Nurse 3 does not work
Stop search after 2 solutions


2

### Shift requests

Nurses sometimes prefer working specific shifts. We can take that into account.

In [91]:
from typing import Union

In [92]:
num_nurses = 5
num_shifts = 3
num_days = 7
all_nurses = range(num_nurses)
all_shifts = range(num_shifts)
all_days = range(num_days)
# the columns are the days, the rows are the nurses, the list inside the rows represent the shifts.
# [0, 0, 1] means the nurse is requesting the third shift on the day.
shift_requests = [
    [[0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1]],
    [[0, 0, 0], [0, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 0, 0], [0, 0, 1]],
    [[0, 1, 0], [0, 1, 0], [0, 0, 0], [1, 0, 0], [0, 0, 0], [0, 1, 0], [0, 0, 0]],
    [[0, 0, 1], [0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 0], [1, 0, 0], [0, 0, 0]],
    [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 0]],
]

In [93]:
model = cp_model.CpModel()

### Variables
The variables are the same as in the first example.

In [94]:
shifts = {}
for n in all_nurses:
    for d in all_days:
        for s in all_shifts:
            shifts[(n, d, s)] = model.NewBoolVar(f"shift_n{n}_d{d}_s{s}")

### Constraints

Constraints 1 and 2 are the same as in the first example.

In [95]:
# Each nurse is only allowed to work one or zero shifts for each day
for n in all_nurses:
    for d in all_days:
        model.AddAtMostOne(shifts[(n, d, s)] for s in all_shifts)
   
# Each shift has exactly one nurse assigned to it.
for d in all_days:
    for s in all_shifts:
        model.AddExactlyOne(shifts[(n, d, s)] for n in all_nurses)

The approach for assigning shifts evenly is also the same.

In [96]:
min_shifts_per_nurse = (num_shifts * num_days) // num_nurses

if num_shifts * num_days % num_nurses == 0:
    max_shifts_per_nurse = min_shifts_per_nurse
else:
    max_shifts_per_nurse = min_shifts_per_nurse + 1
    
for n in all_nurses:
    num_shifts_worked: Union[cp_model.LinearExpr, int] = 0
    for d in all_days:
        for s in all_shifts:
            num_shifts_worked += shifts[(n, d, s)]
    model.Add(min_shifts_per_nurse <= num_shifts_worked)
    model.Add(num_shifts_worked <= max_shifts_per_nurse)

### Objective

This time, we have an objective function.

We go through each request and check if the nurse has requested that shift for that day. If the request matches the solvers value, we essentially calculate 1 * 1. If not, we get 0 * 1 if the shift assigned but not requested, 0 * 0 if the shift is neither requested nor assigned, and 1 * 0 if the shift is requested but not assigned.

So, when maximizing this sum, we reward assigning shifts on request.

In [97]:
model.Maximize(
    sum(
        shift_requests[n][d][s] * shifts[(n, d, s)]
        for n in all_nurses
        for d in all_days
        for s in all_shifts
    )
)

In [98]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [105]:
if status == cp_model.OPTIMAL:
    requests_met_per_nurse = {(n, option) : 0 for n in all_nurses for option in ["met", "not met"]}
    print("Solution:")
    for d in all_days:
        print("Day", d)
        for n in all_nurses:
            for s in all_shifts:
                if solver.Value(shifts[(n, d, s)]) == 1:
                    if shift_requests[n][d][s] == 1:
                        requests_met_per_nurse[(n, "met")] += 1
                        print("Nurse", n, "works shift", s, "(requested).")
                    else:
                        requests_met_per_nurse[(n, "not met")] += 1
                        print("Nurse", n, "works shift", s, "(not requested).")
        print()
    print(
        f"Number of shift requests met = {solver.ObjectiveValue()}",
        f"(out of {num_nurses * min_shifts_per_nurse})",
    )
    print("Requests met per nurse:", requests_met_per_nurse)
else:
    print("No optimal solution found !")

Solution:
Day 0
Nurse 1 works shift 0 (not requested).
Nurse 2 works shift 1 (requested).
Nurse 3 works shift 2 (requested).

Day 1
Nurse 1 works shift 0 (not requested).
Nurse 2 works shift 1 (requested).
Nurse 4 works shift 2 (requested).

Day 2
Nurse 0 works shift 2 (not requested).
Nurse 3 works shift 0 (requested).
Nurse 4 works shift 1 (requested).

Day 3
Nurse 0 works shift 2 (not requested).
Nurse 2 works shift 0 (requested).
Nurse 3 works shift 1 (requested).

Day 4
Nurse 0 works shift 2 (requested).
Nurse 1 works shift 1 (not requested).
Nurse 4 works shift 0 (requested).

Day 5
Nurse 0 works shift 2 (not requested).
Nurse 3 works shift 0 (requested).
Nurse 4 works shift 1 (requested).

Day 6
Nurse 0 works shift 2 (requested).
Nurse 1 works shift 1 (not requested).
Nurse 2 works shift 0 (not requested).

Number of shift requests met = 13.0 (out of 20)
Requests met per nurse: {(0, 'met'): 2, (0, 'not met'): 3, (1, 'met'): 0, (1, 'not met'): 4, (2, 'met'): 3, (2, 'not met'): 1,

### Results
As we can see, we wouldn't want to be nurse 1 😥